In [1]:
# Color space
"""
First we'll use an algorithm to change the color channels from RGB to LAB.
L - lightness
A and B - color spectrums green-red and blue-yellow
f(L) = a b
"""

"\nFirst we'll use an algorithm to change the color channels from RGB to LAB.\nL - lightness\nA and B - color spectrums green-red and blue-yellow\nf(L) = a b\n"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
cd '/content/drive/My Drive/app/colorize&restore-images/'

/content/drive/My Drive/app/colorize&restore-images


In [0]:
# imports
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, xyz2lab, rgb2gray
from skimage.io import imsave
import numpy as np
import os, random
import tensorflow as tf

In [0]:
# get images
image = img_to_array(load_img('woman.jpg'))
image = np.array(image, dtype=float)

In [0]:
X = rgb2lab(1.0/255 * image)[:,:,0]
Y = rgb2lab(1.0/255 * image )[:,:,1:]
Y /= 128

In [8]:
X.shape, Y.shape

((400, 400), (400, 400, 2))

In [0]:
X = X.reshape(1, 400, 400, 1)
Y = Y.reshape(1, 400, 400, 2)

In [0]:
# the network
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))

model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))

model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))

model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))

model.add(UpSampling2D((2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))

model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))

model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [0]:
model.compile(optimizer='rmsprop', loss='mse')

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, None, None, 8)     80        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 8)     584       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 16)    1168      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, None, None, 16)    2320      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, None, None, 32)    4640      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, None, None, 32)    9248      
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, None, None, 32)    0         
__________

In [14]:
history = model.fit(x=X, y=Y, batch_size=1, epochs=1000)

Epoch 1/1000
1/1 [==============================] - 4s 4s/step - loss: 0.4955
Epoch 2/1000
1/1 [==============================] - 0s 26ms/step - loss: 0.5306
Epoch 3/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.1200
Epoch 4/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.0149
Epoch 5/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.0069
Epoch 6/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.0062
Epoch 7/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.0061
Epoch 8/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.0060
Epoch 9/1000
1/1 [==============================] - 0s 20ms/step - loss: 0.0058
Epoch 10/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.0057
Epoch 11/1000
1/1 [==============================] - 0s 20ms/step - loss: 0.0056
Epoch 12/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.0054
Epoch 13/1000
1/1 [====================

In [19]:
print (model.evaluate(X, Y, batch_size=1))

1/1 [==============================] - 0s 64ms/step
0.000371362577425316


In [0]:
output = model.predict(X)
output *= 128

In [23]:
# output colorizations
cur = np.zeros((400, 400, 3))
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]
imsave("img_result.png", lab2rgb(cur))
imsave("img_gray_version.png", rgb2gray(lab2rgb(cur)))

/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))


In [24]:
!ls

AlphaVersion.ipynb  img_gray_version.png  img_result.png  woman.jpg
